### Portfolio Optimisation and the Efficient Frontier


### Modern Portfolio Theory

Modern portfolio theory (also popularly called Mean-Variance Portfolio Theory (MVP)) is a major breakthrough in finance. It is based on the premise that returns are normally distributed and by looking at mean and variance, we can essentially describe the distribution of end-of-period wealth.

The basic idea of this theory is to achieve diversification by constructing a portfolio for a minimal portfolio risk or maximal portfolio returns. Accordingly, the Efficient Frontier is a set of optimal portfolios in the risk-return spectrum, and portfolios located under the Efficient Frontier curve are considered sub-optimal.

This means that the portfolios on the frontier offers

- Highest expected return for a given level of risk

- Lowest level of risk for a given level of expected returns

In essence, the investors' goal should be to select a level of risk that he/she is comfortable with and then find a portfolio that maximizes returns based on the selected risk level.

### Import Libraries


In [1]:
# Base libs
from numpy import *
from numpy.linalg import multi_dot
import pandas as pd
import yfinance as yf

#cufflinks
import cufflinks as cf
cf.set_config_file(offline = True, dimensions=((1000,600)), theme = 'henanigans')

#Plotly Express
import plotly.express as px
px.defaults.template = 'plotly_dark'
px.defaults.width = 1000
px.defaults.height = 600

#warnings
import warnings
warnings.filterwarnings('ignore')

We will retrieve some Nasdaq stocks as previously, to build our portfolio

In [2]:
#Nasdaq-listed stocklist:
symbols = ['AMD', 'CSCO', 'INTC', 'INTU', 'NVDA']

#number of assets
numofasset = len(symbols)

#Number of portfolios for optimisation
numofportfolios = 5000

### Retrieve the data

Retrieve the data from yfinance - note we're really only interested in the Adj Close column now

In [3]:
#Yahoo finance data for the last 6 years
#nasdaqstocks = yf.download(symbols, start = '2017-10-01', end = '2021-12-31', progress=False)['Adj Close']

#write to local storage in case we need it later
#nasdaqstocks.to_csv('data/nasdaqstocks.csv')

#Load locally-stored data:
df = pd.read_csv('data/nasdaqstocks.csv', index_col=0, parse_dates=True)

#Have a shufty at the output
df

,AMD,CSCO,INTC,INTU,NVDA
Date,,,,,
2017-10-02,12.710000,29.488768,34.863285,137.060516,44.301590
2017-10-03,13.420000,29.576143,35.166912,137.166870,44.393162
2017-10-04,13.310000,29.470388,35.131195,137.959778,44.764404
2017-10-05,13.340000,29.602577,35.300861,138.143494,44.739658
2017-10-06,13.230000,29.743589,35.390163,139.801544,44.870834
...,...,...,...,...,...
2021-12-23,146.139999,61.915154,50.922092,634.972839,296.349487
2021-12-27,154.360001,63.048477,51.547329,652.013062,309.397308
2021-12-28,153.149994,63.157833,51.368690,649.026550,303.168335


### Visualise price history

In [4]:
#Plot price history
df.normalize().iplot(kind='line', title = 'Normalised Price Plot')

### Calculate Returns

In [5]:
# Calculate returns
returns = df.pct_change().fillna(0)
returns.tail()

,AMD,CSCO,INTC,INTU,NVDA
Date,,,,,
2021-12-23,0.015707,0.012189,0.006671,0.006412,0.008163
2021-12-27,0.056247,0.018304,0.012278,0.026836,0.044028
2021-12-28,-0.007839,0.001734,-0.003466,-0.004580,-0.020133
2021-12-29,-0.031929,0.006768,0.001352,-0.002693,-0.010586
2021-12-30,-0.020977,-0.005316,-0.001736,-0.007206,-0.013833


In [6]:
#Plot annualised return and vol

pd.DataFrame(
    {
        'Annualised Return': round(returns.mean() * 252 * 100 * 2),
        'Annualised vol': round(returns.std() * sqrt (252), 2)
    }
).iplot(kind='bar', title='Annualised Returns and Vol (%) for all stocks', shared_xaxes = True, subplots=True)

### Portfolio statistics

Consider a portfolio which is fully invested in risky assets. Let ${w}$ and ${μ}$ be the vector of weights and mean returns of n assets. 

$$\ {w=}\left( 
\begin{array}{c}
w_1 \\
w_2 \\
\vdots \\
w_n \\ 
\end{array}%
\right);
\ \mathbf{\mu=}\left( 
\begin{array}{ccc}
\mu_1 \\ 
\mu_2 \\ 
\vdots \\
\mu_n \\ 
\end{array}%
\right)$$

Where $\sum_{i=1}^{n}w_i=1$

**Expected Portfolio Return** is then the dot product of the expected returns for each asset ${μ}$, and their respective portfolio weights ${w}$ 

**Expected Portfolio Variance** is the dot product (implemented below via multidot) of the weights and the covariance matrix:

$$\sigma^2_\pi = w^T\cdot\Sigma\cdot w$$

Where ${\Sigma}$ is the covariance matrix:
$${\sigma=}\left( 
\begin{array}{ccc}
\sigma_{1,1} & \dots & \sigma_{1,n} \\ 
\vdots & \ddots & \vdots  \\ 
\sigma_{n,1} & \dots & \sigma_{n,n} \\ %
\end{array}%
\right)$$



### An Equally-weighted Portfolio

Firstly, let's look at a portfolio where each asset is weighted equally.  Let's calculate the portfolio stats:

In [7]:
#define the weights for the portfolio
wts = numofasset * [1./numofasset]
array(wts).shape

(5,)

In [8]:
#Oh no, the dreaded rank-1 array!  Kill it now
wts = array(wts)[:, newaxis]
wts

array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])

In [9]:
#check it's now a proper vector now
wts.shape

(5, 1)

Asset returns

In [10]:
#calculate the asset returns
ret = array(returns.mean() * 252)[:, newaxis]
ret

array([[0.72568637],
       [0.22075064],
       [0.16066391],
       [0.41706877],
       [0.56235141]])

In [11]:
#check shape :)
ret.shape

(5, 1)

Portfolio return
Now let's calculate the portfolio return - this is $\mu_\pi = w^T\cdot\mu$

In [12]:
wts.T @ ret

array([[0.41730422]])

### Portfolio Vol


In [13]:
#covariance matrix - comes straight out of, what, Pandas?
#Presumably it's shown as daily, which is why we multiply by 252

cov = returns.cov() * 252
var = multi_dot([wts.T, cov, wts])

print(f'Portfolio vol: {sqrt(var)}')


print('Covariance matrix:')
cov


Portfolio vol: [[0.32154382]]
Covariance matrix:


,AMD,CSCO,INTC,INTU,NVDA
AMD,0.305518,0.066324,0.079551,0.088370,0.173148
CSCO,0.066324,0.081784,0.061835,0.055460,0.070675
INTC,0.079551,0.061835,0.137862,0.065989,0.099779
INTU,0.088370,0.055460,0.065989,0.106507,0.101640
NVDA,0.173148,0.070675,0.099779,0.101640,0.227546


In [14]:
returns

,AMD,CSCO,INTC,INTU,NVDA
Date,,,,,
2017-10-02,0.000000,0.000000,0.000000,0.000000,0.000000
2017-10-03,0.055862,0.002963,0.008709,0.000776,0.002067
2017-10-04,-0.008197,-0.003576,-0.001016,0.005781,0.008363
2017-10-05,0.002254,0.004485,0.004830,0.001332,-0.000553
2017-10-06,-0.008246,0.004764,0.002530,0.012002,0.002932
...,...,...,...,...,...
2021-12-23,0.015707,0.012189,0.006671,0.006412,0.008163
2021-12-27,0.056247,0.018304,0.012278,0.026836,0.044028
2021-12-28,-0.007839,0.001734,-0.003466,-0.004580,-0.020133


### A Randomly Weighted Portfolio

We will now implement a Monte Carlo simulation to generate random portfolio weightings, and calculate the associated Portfolio Return, Variance and Sharpe Ratio for every simulation.  We will then identify the portfolio with the highest return per unit of risk

In [15]:
def portfolio_simulation(returns):

    #initialise lists
    rets = []
    vols = []
    wts = []

    #siumulate 5,000 portfolios:
    for i in range(numofportfolios):
        
        #Generate random weights for this portfolio
        weights = random.random(numofasset)[:, newaxis]

        #But we need to make sure this random allocation totals up to 1!
        weights /= sum(weights)

        #Portfolio stats:
        rets.append(
            weights.T @ array(returns.mean() * 252)[:, newaxis]
        )
        vols.append(
            sqrt(multi_dot([weights.T, returns.cov()*252, weights]))
        )
        wts.append(
            weights.flatten()
        )

    #DataFrame to store the results
    portdf = 100 * pd.DataFrame({
        'port_rets': array(rets).flatten(),
        'port_vols': array(vols).flatten(),
        'weights': list(array(wts))
    })

    #calculate the sharpe ratio for each portfolio:
    portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']

    return round(portdf, 2)


In [16]:
#Now let's see how this works with our Nasdaq stocks:
temp = portfolio_simulation(returns)
temp.head()

,port_rets,port_vols,weights,sharpe_ratio
0,33.51,30.00,"[8.99846763567328, 30.277973922636576, 29.4623...",1.12
1,47.05,34.95,"[28.589313934007937, 22.364888417803368, 11.64...",1.35
2,40.58,30.04,"[25.354986448044343, 37.25986370129388, 7.6136...",1.35
3,38.53,31.29,"[3.274821912401069, 24.501330685582186, 15.207...",1.23
4,36.27,31.71,"[8.778443140900578, 24.856073917893127, 28.403...",1.14


In [17]:
#Now we know that the most efficient portfolio is the one with the highest sharpe ratio.  Let's get that:

temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets                                                   54.57
port_vols                                                   36.64
weights         [42.041828102607504, 4.252949240413225, 0.8667...
sharpe_ratio                                                 1.49
Name: 1248, dtype: object

In [18]:
#Check this by looking at the descriptive statistics - look at the max column
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,5000.0,41.610358,5.918657,19.51,37.7,41.55,45.55,66.22
port_vols,5000.0,32.829296,2.622068,26.97,31.0,32.58,34.42,48.99
sharpe_ratio,5000.0,1.263318,0.109429,0.65,1.2,1.28,1.34,1.49


In [19]:
#Get the weights associated with the  max sharpe ratio
msrpwts = temp['weights'][temp['sharpe_ratio'].idxmax()]

#zip 'em up alongside the associated weights:
dict(zip(symbols, around(msrpwts)))

{'AMD': 42.0, 'CSCO': 4.0, 'INTC': 1.0, 'INTU': 46.0, 'NVDA': 7.0}

### Visualise the Portfolios (and the most efficient one)

In [20]:
#plot simulated portfolios

fig = px.scatter(
    temp,
    x = 'port_vols',
    y = 'port_rets',
    color = 'sharpe_ratio',
    labels = {
        'port_vols': 'Expected Volatility',
        'port_rets': 'Expected Return',
        'sharpe_ratio':  'Sharpe Ratio'
    },
    title = 'Monte Carlo Simulated Portfolio',
    color_continuous_scale=px.colors.sequential.Rainbow
   # color_continuous_scale=px.colors.sequential.Rainbow[::-1] #Note the [::-1] REVERSES the scale
).update_traces(mode='markers', marker = dict(symbol='circle'))

fig.add_scatter(
    mode='markers',
    x = [temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']],
    y = [temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']],
    marker = dict(color='white', size = 20, symbol = 'star', line = dict(color='MediumPurple', width=1)),
    name = 'Portfolio with Max Sharpe Ratio'
).update(layout_showlegend = False)

#Show spikes
#Oh right, these are the horizontal / vertical bars that follow your mouse.  Nice!
fig.update_xaxes(showspikes = True)
fig.update_yaxes(showspikes = True)

fig.show()

### The Efficient Frontier

The Efficient Frontier is formed by a set of portfolios offering the highest expected returns for a certain volatility, or the lowest volatility for a certain level of expected return

We may express these in terms of a **Return Objective** or a **Risk Constraint**:

**Return Objective**

$${\underset{w_1,w_2,\dots,w_n}{minimize} \space\space \sigma^2_{p}(w_1,w_2,\dots,w_n)}$$

Subject to

$${E[R_p] = m}$$

**Risk Constraint**

$${\underset{w_1,w_2,\dots,w_n}{maximize} \space\space E[R_p(w_1,w_2,\dots,w_n)]}$$

Subject to

$${\sigma^2_{p}(w_1,w_2,\dots,w_n)=v^2}$$

where $\sum_{i=1}^{n}w_i=1$ for the above objectives

We can use numerical optimisation techniques to achieve this objective.  The goal of optimisation is to find the optimal value of the objective function, by adjusting the target variables operating within some boundary conditions and constraints (such as, above, the Return Objective / Risk Constraint and the budget equation)

### Constrained Optimisation

Constructio of an optimal portfolio is a **constrained optimisation** problem - where we specify some boundary conditions and constraints.  The **objective function** here is a function returning maximum Sharpe Ratio, minimum variance (volatility), and the target variables that we will be adjusting are the **portfolio weights**

We will use the ```minimise``` function from the ```scipy``` optimisation module to achieve our objective:

``` python 
sco.minimize(
    fun, 
    x0, 
    args=(), 
    method=None, 
    jac=None, 
    hess=None, 
    hessp=None, 
    bounds=None, 
    constraints=(), 
    tol=None, 
    callback=None, 
    options=None
    )
```

In [21]:
#import optimisation module from scipy
import scipy.optimize as sco

**Portfolio Statistics function**

Let's pull together some key elements that we're interested in into one single function

In [22]:
#Define Portfolio statistics function
def portfolio_stats(weights):
    
    weights = array(weights)[:,newaxis]
    port_rets = weights.T @ array(returns.mean() * 252)[:,newaxis]    
    port_vols = sqrt(multi_dot([weights.T, returns.cov() * 252, weights])) 
    
    return array([port_rets, port_vols, port_rets/port_vols]).flatten()

**Example 1:  Finding the portfolio with the maximum Sharpe Ratio**

Question - are we MINIMISING IT (as per the name of the function) or MAXIMISING it?  Who knows

In [23]:
#Maximising (or minimising??) sharpe ratio
def min_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2]

In [24]:
# I don't know why we're doing this so i'm ignoring it
# Each asset boundary ranges from 0 to 1
# tuple((0, 1) for x in range(numofasset))

Specify constraints and bounds

In [25]:
cons = (
    {
        'type': 'eq',
        'fun': lambda x: sum(x) - 1 #ie the sum of all asset weights must equal 1
    }
)

#Now, it seems you have to pass in a set of boundaries - presumably this is so scipy knows to not
#go past particular values.  Check this!  But basically it seems each asset can have 
#min allocation 0, max 1
bnds = tuple((0,1) for x in range(numofasset))

initial_wts = numofasset*[1/numofasset]
initial_wts

[0.2, 0.2, 0.2, 0.2, 0.2]

Now, the actual optimisation bit:

In [26]:
opt_sharpe = sco.minimize(
    min_sharpe_ratio, 
    initial_wts,
    method = 'SLSQP',
    bounds = bnds,
    constraints = cons
)

opt_sharpe

     fun: -1.5025592207321696
     jac: array([-3.00854445e-05,  8.81428272e-02,  4.00766119e-01,  2.02655792e-05,
       -1.20997429e-05])
 message: 'Optimization terminated successfully'
    nfev: 42
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([3.72075085e-01, 1.45825192e-17, 0.00000000e+00, 5.80673348e-01,
       4.72515672e-02])

The information here is quite cryptic, but it seems that the optimum weights are in the 'x' attribute:

In [27]:
#Optimum portfolio weights
list(
    zip(
        symbols,
        around(opt_sharpe['x']*100, 2)
    )
)

[('AMD', 37.21), ('CSCO', 0.0), ('INTC', 0.0), ('INTU', 58.07), ('NVDA', 4.73)]

In [28]:
#Portfolio stats
stats = ['Returns','Vol','Sharpe Ratio']
jim = list(
    zip(
        stats,
        around(portfolio_stats(opt_sharpe['x']),4)
    )
)

In [29]:
jim[1][1]

0.3586

Now let's present this on a chart.  The below code is identical to what we used above, with the exception that the 'star' is now our optimum portfolio.  I'm assuming it'll be in the same place as previously (more or less) but let's see

In [30]:
#plot simulated portfolios

fig = px.scatter(
    temp,
    x = 'port_vols',
    y = 'port_rets',
    color = 'sharpe_ratio',
    labels = {
        'port_vols': 'Expected Volatility',
        'port_rets': 'Expected Return',
        'sharpe_ratio':  'Sharpe Ratio'
    },
    title = 'Monte Carlo Simulated Portfolio',
    color_continuous_scale=px.colors.sequential.Rainbow
   # color_continuous_scale=px.colors.sequential.Rainbow[::-1] #Note the [::-1] REVERSES the scale
).update_traces(mode='markers', marker = dict(symbol='circle'))

fig.add_scatter(
    mode='markers',
    x = [jim[1][1]*100],
    y = [jim[0][1]*100],
    marker = dict(color='white', size = 20, symbol = 'star', line = dict(color='MediumPurple', width=1)),
    name = 'Portfolio Optimised by scipy'
).update(layout_showlegend = False)

fig.add_scatter(
    mode='markers',
    x = [temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']],
    y = [temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']],
    marker = dict(color='Green', size = 20, symbol = 'star', line = dict(color='MediumPurple', width=1)),
    name = 'Portfolio with Maximum Sharpe Ratio'
).update(layout_showlegend = False)

#Show spikes
#Oh right, these are the horizontal / vertical bars that follow your mouse.  Nice!
fig.update_xaxes(showspikes = True)
fig.update_yaxes(showspikes = True)

fig.show()

### Example 2:  Finding the Global Minimum Variance Portfolio

In [31]:
# Minimise variance
def min_variance (weights):
    # Presumably what's returned from portfolio_stats is actually the standard deviation
    # which is why we're squaring it

    #This really should just be called 'get variance' as it does nothign to attempt to 'minimise' anything
    
    return portfolio_stats(weights)[1]**2

In [32]:
#optimising for minimum variance
opt_var = sco.minimize(
    min_variance,
    initial_wts,
    method = 'SLSQP',
    bounds = bnds,
    constraints = cons
)

opt_var

     fun: 0.07173501736681738
     jac: array([0.14868557, 0.14347099, 0.14346611, 0.14346963, 0.16627645])
 message: 'Optimization terminated successfully'
    nfev: 54
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([4.68620577e-18, 5.90648509e-01, 1.14052174e-01, 2.95299317e-01,
       0.00000000e+00])

In [33]:
list(
    zip(
        symbols,
        around(opt_var['x']*100,2)
    )
)

[('AMD', 0.0),
 ('CSCO', 59.06),
 ('INTC', 11.41),
 ('INTU', 29.53),
 ('NVDA', 0.0)]

In [34]:
jim = list(
    zip(
        stats,
        around(portfolio_stats(opt_var['x']),4)
    )
)

print(jim)

[('Returns', 0.2719), ('Vol', 0.2678), ('Sharpe Ratio', 1.0151)]


Now let's view this on a chart

In [35]:
#plot simulated portfolios

fig = px.scatter(
    temp,
    x = 'port_vols',
    y = 'port_rets',
    color = 'sharpe_ratio',
    labels = {
        'port_vols': 'Expected Volatility',
        'port_rets': 'Expected Return',
        'sharpe_ratio':  'Sharpe Ratio'
    },
    title = 'Monte Carlo Simulated Portfolio',
    color_continuous_scale=px.colors.sequential.Rainbow
   # color_continuous_scale=px.colors.sequential.Rainbow[::-1] #Note the [::-1] REVERSES the scale
).update_traces(mode='markers', marker = dict(symbol='circle'))

fig.add_scatter(
    mode='markers',
    x = [jim[1][1]*100],
    y = [jim[0][1]*100],
    marker = dict(color='white', size = 20, symbol = 'star', line = dict(color='MediumPurple', width=1)),
    name = 'GMVP'
).update(layout_showlegend = False)

#Show spikes
#Oh right, these are the horizontal / vertical bars that follow your mouse.  Nice!
fig.update_xaxes(showspikes = True)
fig.update_yaxes(showspikes = True)

fig.show()

### Example 3:  Finding the Efficient Frontier

So here, what I think we're doing is to identify a set of target returns - then find the portfolios that have the minimum variance associated with that return (I think)

In [36]:
# minimising volatility
def min_volatility(weights):
    return portfolio_stats(weights)[1]

In [37]:
#Efficient Frontier parameters
targetrets = linspace(0.2, 0.6, 100)  #target returns
tvols = []

for tr in targetrets:

    ef_cons = (
        {
            #here I think we're trying to optimise for returns (i.e. portfolio_stats(...)[0] = target return)
            'type': 'eq',
            'fun': lambda x: portfolio_stats(x)[0] - tr 
        },
        {
            'type': 'eq',
            'fun': lambda x: sum(x) - 1 #ie. the sum of all asset weights must = 1
        }
    )

    opt_ef = sco.minimize(
        min_volatility,
        initial_wts,
        method = 'SLSQP',
        bounds = bnds,
        constraints = ef_cons
    )

    #print (opt_ef)

    tvols.append(opt_ef['fun']) #wtf if this?


targetvols = array(tvols)

In [38]:
# Dataframe for EF
efport = pd.DataFrame({
    'targetrets' : around(100*targetrets,2),
    'targetvols': around(100*targetvols,2),
    'targetsharpe': around(targetrets/targetvols,2)
})

efport.head()

,targetrets,targetvols,targetsharpe
0,20.00,28.19,0.71
1,20.40,27.95,0.73
2,20.81,27.81,0.75
3,21.21,27.69,0.77
4,21.62,27.57,0.78


In [39]:
#Now visualise it

# Plot efficient frontier portfolio
fig = px.scatter(
    efport, x='targetvols', y='targetrets',  color='targetsharpe',
    labels={'targetrets': 'Expected Return', 'targetvols': 'Expected Volatility','targetsharpe': 'Sharpe Ratio'},
    title="Efficient Frontier Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))


# Plot maximum sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_sharpe['x'])[1]], 
    y=[100*portfolio_stats(opt_sharpe['x'])[0]],
    marker=dict(color='red', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Plot minimum variance portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_var['x'])[1]], 
    y=[100*portfolio_stats(opt_var['x'])[0]],
    marker=dict(color='green', size=20, symbol='star'),
    name = 'Min Variance'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()